### Finetuning a transformer model for a domain with `unsloth`

*This notebook must be run on a GPU system with Cuda drivers.*

----

Install `unsloth`. It makes finetuning of transformer models much less resource-intensive. Read more [here](https://huggingface.com/unsloth) and [here](https://unsloth.ai).

In [ ]:
!pip install unsloth
# also get the latest build of unsloth from GitHub
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Import libraries.

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

AssertionError: Torch not compiled with CUDA enabled

Check over on [unsloth's huggingface profile](https://huggingface.com/unsloth) which models they support. In this case we chose "`unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit`".

We download the model. Some notes:

- Unsloth supports [RoPE scaling](https://paperswithcode.com/paper/roformer-enhanced-transformer-with-rotary), meaning that the `max_seq_length` can be set much higher than what the model was initially trained on (if one would want to).
- Set `dtype = None` for auto detection.
- Loading in `4bit` means that the model's weights are quantized to 4 bits, which significantly reduces the memory usage and computational requirements compared to using full precision (usually 16 or 32 bits).


In [ ]:
model_name =  "unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit"
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2024.11.9: Fast Qwen2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add [LoRA adapters](https://arxiv.org/abs/2106.09685) so we only need to update 1 to 10% of all parameters.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

**Finetuning data**: Provide a jsonl file formatted as the example file `domain.json`.

In [ ]:
finetuning_file = "domain.jsonl"
dataset = load_dataset("json", data_files=finetuning_file, split="train")

# Inspect the dataset
print(dataset)
print(dataset[0])  # View the first example

Dataset({
    features: ['text'],
    num_rows: 500
})
{'text': 'Traditional Swedish meatballs are made with a mix of beef and pork.'}


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "fine_tuned_model",
        report_to = "none", # Use this for WandB etc
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.406 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 8,798,208


Step,Training Loss
1,4.474100
2,4.511800
3,4.322800
4,3.761900
5,4.250400
6,4.135600
7,3.773900
8,3.154400
9,3.171700
10,2.930700


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

108.2142 seconds used for training.
1.8 minutes used for training.
Peak reserved memory = 1.057 GB.
Peak reserved memory for training = 0.391 GB.
Peak reserved memory % of max memory = 7.167 %.
Peak reserved memory for training % of max memory = 2.651 %.


### Use the fine-tuned model

Use the finetuned model for text summarization, and characterization.

We define the `big_chunk_of_text` to be summarized and described.

The `big_chunk_of_text` to be summarized and described must fit within the total token limit defined by `max_seq_length` earlier. This limit includes all tokens: the system message, user prompt, input text, and the generated output. Since the model generates additional tokens during summarization (controlled by max_new_tokens), a margin must be reserved for the output. Note that token counts differ from word counts, as tokenization can split a single word into multiple tokens.

In [ ]:
# Using cop_key_clusters as test data, extract all tweets belonging to one of the clusters
df = pd.read_csv('cop_key_clusters.csv', on_bad_lines='skip', low_memory= False)
text_list = df[df['cluster'] == 799]['text'].tolist()
text_list = list(set(text_list)) # remove duplicates


In [ ]:
# Define the content strings as variables
system_message = "You are a social science researcher skilled at qualitatively assessing the core meanings of text."
user_prompt = (
    "Summarize the following text with a particular focus on its key unifying themes, ideological stance, and sentiment. "
    "Your only output should be bullet points pertaining to that summary task. Here is the text to be summarized:\n"
    f"{big_chunk_of_text}"
)

In [ ]:
# Prepare the summarization prompt with the variables
messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "user",
        "content": user_prompt,
    },
]

# Generate the summary (from the rest of the code)
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()

# Dynamically construct the removal string
removal_string = f"system\n{system_message}\nuser\n{user_prompt}"

# Clean the output
clean_summary = decoded_output.replace(removal_string, "").strip().lstrip('assistant').strip()


# Print only the clean summary
print(clean_summary)

Key unifying themes:
- Nutritional value (tender meatballs)
- Cultural significance (traditional Swedish meatballs)
- Religious symbolism (meatball symbolism for Sweden)

Ideological stance:
- Anti-globalization
- Nordic nationalism

Sentiment:
- Appreciation for Swedish food traditions
- Joyful celebration of holiday festivities
- Gratitude towards Swedish heritage


In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit")
model = AutoModelForCausalLM.from_pretrained("unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit")

# Define the content strings
system_message = "You are a social science researcher skilled at qualitatively assessing the core meanings of text."

# Function to prepare the prompt
def prepare_prompt(chunk):
    user_prompt = (
        "Summarize the following text with a particular focus on its key unifying themes, ideological stance, and sentiment. "
        "Your only output should be bullet points pertaining to that summary task. Here is the text to be summarized:\n"
        f"{chunk}"
    )
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
    ]

# Function to clean the output
def clean_output(decoded_output, user_prompt):
    removal_string = f"system\n{system_message}\nuser\n{user_prompt}"
    return decoded_output.replace(removal_string, "").strip().lstrip("assistant").strip()

# Function to generate summary for a chunk
def generate_summary(chunk):
    messages = prepare_prompt(chunk)
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to("cuda")
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=100,  # Adjust as needed
        use_cache=True,
    )
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return clean_output(decoded_output, messages[1]["content"])

# Function to chunk the text
def chunk_text(text, max_tokens):
    tokens = tokenizer(text, return_tensors="pt")["input_ids"][0]
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# Process big chunk of text

max_seq_length = tokenizer.model_max_length  # Use tokenizer's max model length
# If RoPE scaling allows, manually set a higher limit
max_seq_length = 2048  # Adjust if required for your model and hardware
max_tokens_per_chunk = max_seq_length - 200  # Reserve space for prompt and output

chunks = chunk_text(big_chunk_of_text, max_tokens_per_chunk)

# Generate sub-summaries for each chunk
sub_summaries = [generate_summary(chunk) for chunk in chunks]

# Combine sub-summaries and generate overarching summary
combined_text = " ".join(sub_summaries)
final_summary = generate_summary(combined_text)

# Print the final summary
print("Final Summary:")
print(final_summary)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


AttributeError: 'Qwen2ForCausalLM' object has no attribute 'max_seq_length'